# A Simple Convolutional Neural Network(Using GPU)
![gChvH4](https://gitee.com/pxqp9W/testmarkdown/raw/master/imgs/2020/07/gChvH4.png)
![VBcUKI](https://gitee.com/pxqp9W/testmarkdown/raw/master/imgs/2020/07/VBcUKI.png)

In [8]:
import torch 
from torchvision import transforms 
from torchvision import datasets 
from torch.utils.data import DataLoader 
import torch.nn.functional as F 
import torch.optim as optim
import time

## 1、Prepare Dataset

In [9]:
batch_size = 64

## Convert the PIL Image to Tensor.
transform = transforms.Compose([
    transforms.ToTensor(), # 图像转变为ToTensor，取值变为0-1
    transforms.Normalize((0.1307, ), (0.3081, )) 
    # 0.1307是均值mean, 0.3081是标准差std
    # 这两个数是根据 MNIST数据集的特征得到的经验值
])

## 构造loader
train_dataset = datasets.MNIST(root='./dataset/mnist/', 
                               train=True, 
                               download=True, 
                               transform=transform) # transform放入到了数据集中，这样每次取出来的数据都会直接做变换
train_loader = DataLoader(train_dataset, 
                          shuffle=True, 
                          batch_size=batch_size)
test_dataset = datasets.MNIST(root='./dataset/mnist/' , 
                              train=False, 
                              download=True, 
                              transform=transform)
test_loader = DataLoader(test_dataset, 
                         shuffle=False, 
                         batch_size=batch_size)

## 2、Design CNN Model 

In [10]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__() 
        self.conv1 = torch.nn.Conv2d(1, 10, kernel_size=5) 
        self.conv2 = torch.nn.Conv2d(10, 20, kernel_size=5) 
        self.pooling = torch.nn.MaxPool2d(2) 
        self.fc = torch.nn.Linear(320, 10)
    
    def forward(self, x):
        # Flatten data from (n, 1, 28, 28) to (n, 784)
        batch_size = x.size(0) 
        x = F.relu(self.pooling(self.conv1(x))) 
        x = F.relu(self.pooling(self.conv2(x)))
        x = x.view(batch_size, -1) # flatten 
        x = self.fc(x)  # 因为后面要用PyTorch的交叉熵函数，因此最后一层不做激活
        return x
model = Net()
# Define device as the first visible cuda device if we have CUDA available.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
# Convert parameters and buffers of all modules to CUDA Tensor.
model.to(device)

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (pooling): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=320, out_features=10, bias=True)
)

## 3、Construct Loss and Optimizer

In [11]:
criterion = torch.nn.CrossEntropyLoss() 
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5) # momentum表示动量

## 4、Train & Test

In [12]:
## 将训练过程封装到函数
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        inputs, target = data 
        # Send the inputs and targets at every step to the GPU.
        inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()
        
        # forward + backward + update
        outputs = model(inputs) 
        loss = criterion(outputs, target) 
        loss.backward() 
        optimizer.step()
        running_loss += loss.item() 
        
        if batch_idx % 300 == 299: 
            print('[%d, %5d] loss: %.3f' % (epoch + 1, batch_idx + 1, running_loss / 300)) 
            running_loss = 0.0

In [13]:
## 将测试过程也封装为函数
def test():
    correct = 0 
    total = 0
    with torch.no_grad(): # Test过程中不需要计算梯度，可以使用“with torch.no_grad()”
        for data in test_loader:
            images, labels = data 
            # Send the inputs and targets at every step to the GPU.
            images, labels = images.to(device), labels.to(device)
            outputs = model(images) 
            _, predicted = torch.max(outputs.data, dim=1) # torch.max用来找行中的最大值所在下标 dim1表示沿着第1个维度找（行是第0个维度，列是第1个维度）
            total += labels.size(0) # labels是一个N*1的向量，取size()得到的是元组(N,1),size(0)返回的是N
            correct += (predicted == labels).sum().item() # 预测正确的数量，predicted == labels进行的是张量之间的比较运算
    print('Accuracy on test set: %d %%' % (100 * correct / total))

## Main Function

In [14]:
if __name__ == '__main__':
    start = time.time()
    for epoch in range(10):
        train(epoch) 
        test()
    end = time.time()
    print("Total time: ", end - start , "s")

[1,   300] loss: 0.650
[1,   600] loss: 0.192
[1,   900] loss: 0.142
Accuracy on test set: 96 %
[2,   300] loss: 0.112
[2,   600] loss: 0.098
[2,   900] loss: 0.084
Accuracy on test set: 97 %
[3,   300] loss: 0.078
[3,   600] loss: 0.072
[3,   900] loss: 0.070
Accuracy on test set: 98 %
[4,   300] loss: 0.064
[4,   600] loss: 0.063
[4,   900] loss: 0.059
Accuracy on test set: 98 %
[5,   300] loss: 0.054
[5,   600] loss: 0.049
[5,   900] loss: 0.058
Accuracy on test set: 98 %
[6,   300] loss: 0.047
[6,   600] loss: 0.047
[6,   900] loss: 0.051
Accuracy on test set: 98 %
[7,   300] loss: 0.045
[7,   600] loss: 0.045
[7,   900] loss: 0.043
Accuracy on test set: 98 %
[8,   300] loss: 0.042
[8,   600] loss: 0.036
[8,   900] loss: 0.044
Accuracy on test set: 98 %
[9,   300] loss: 0.039
[9,   600] loss: 0.036
[9,   900] loss: 0.038
Accuracy on test set: 98 %
[10,   300] loss: 0.035
[10,   600] loss: 0.033
[10,   900] loss: 0.039
Accuracy on test set: 98 %
Total time:  75.21362042427063 s
